In [ ]:
#!/usr/bin/python
# -*- coding: latin-1 -*-
"""This notebook builds the CSV-like file describing the WLF festivals in Spain"""

import os

try :
    import pywikibot as pb
except :
    import sys, inspect
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[0:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb

import mwparserfromhell as mwh

import pandas as pd

from datetime import datetime
from io import StringIO, BytesIO

In [ ]:
from modules.wmtools import upload_to_commons, epoch_time, dms2dd

In [ ]:
BASE_WLF2016_NAME = 'Commons:Wiki Loves Folk/{0}'.format(2016)
FESTIVAL_DB_PAGE  = BASE_WLF2016_NAME + '/Festival DB'

commons_site   = pb.Site("commons", "commons")
wikipedia_site = pb.Site("es", "wikipedia")
wikidata_site  = pb.Site("wikidata", "wikidata")

In [ ]:
def epoch_time(timestamp):
    tdelta = timestamp - datetime.utcfromtimestamp(0)
    return int(tdelta.total_seconds()*1000)

In [ ]:
annexes = [
    ['Anexo:Fiestas de interés turístico de Andalucía', 'Andalusia'],
    ['Anexo:Fiestas de interés turístico de Aragón', 'Aragon'],
    ['Anexo:Fiestas de interés turístico de Asturias', 'Asturias'],
    ['Anexo:Fiestas de interés turístico de Cantabria', 'Cantabria'],
    ['Anexo:Fiestas de interés turístico de Castilla-La Mancha', 'Castile-La Mancha'],
    ['Anexo:Fiestas de interés turístico de Castilla y León', 'Castile and León'],
    ['Anexo:Fiestas de interés turístico de Cataluña', 'Catalonia'],
    ['Anexo:Fiestas de interés turístico de la Comunidad de Madrid', 'Community of Madrid'],
    ['Anexo:Fiestas de interés turístico de la Comunidad Valenciana', 'Valencian Community'],
    ['Anexo:Fiestas de interés turístico de Extremadura', 'Extremadura'],
    ['Anexo:Fiestas de interés turístico de las Islas Baleares', 'Balearic Islands'],
    ['Anexo:Fiestas de interés turístico de las Islas Canarias', 'Canary Islands'],
    ['Anexo:Fiestas de interés turístico de Galicia', 'Galicia'],
    ['Anexo:Fiestas de interés turístico de La Rioja', 'La Rioja'],
    ['Anexo:Fiestas de interés turístico de Navarra', 'Navarre'],
    ['Anexo:Fiestas de interés turístico de la Región de Murcia', 'Region of Murcia'],
    ['Anexo:Fiestas y tradiciones del País Vasco', 'Basque Country']
]

In [ ]:
festivals_df = pd.DataFrame(
    columns=['name', 'aut_com', 'wikidata_id', 'wikidata_timestamp', 'category', 'cat_timestamp', 'image', 'latitude', 'longitude'])

for annex in annexes:
    pb.output("Processing annex " + annex[0])
    page = pb.Page(wikipedia_site, annex[0])
    text = page.text
    wikicode = mwh.parse(text)
    templates = wikicode.filter_templates()

    for template in templates:
        if template.name.lower().strip() == "fila wlf":
            df_row = dict()
            df_row = {
                'name': '',
                'aut_com': annex[1],
                'wikidata_id': '',
                'wikidata_timestamp': '',
                'category': '',
                'cat_timestamp': '',
                'image': '',
                'latitude': 0,
                'longitude': 0
            }
            lat = []
            long = []
            if template.get("coord").value and len(template.get("coord").value) > 2:
                coord_content = template.get("coord").value.strip()
                wikicode = mwh.parse(coord_content)
                coord_templates = wikicode.filter_templates()
                coordinates = [i for i in coord_templates if (i.name.lower().strip() == 'coord') or (i.name.lower().strip() == 'coordenadas')]
                for coordinate in coordinates:
                    #print (coordinate.params)
                    if len(coordinate.params) == 9:
                        lat.append(dms2dd (coordinate.get(1).value.strip(),
                                       coordinate.get(2).value.strip(),
                                       coordinate.get(3).value.strip(),
                                       direction=coordinate.get(4).value.strip()
                                      ))
                        long.append(dms2dd (coordinate.get(5).value.strip(),
                                       coordinate.get(6).value.strip(),
                                       coordinate.get(7).value.strip(),
                                       direction=coordinate.get(8).value.strip()
                                      ))
                    elif len(coordinate.params) == 7:
                        lat.append(dms2dd (coordinate.get(1).value.strip(),
                                       coordinate.get(2).value.strip(),
                                       0,
                                       direction=coordinate.get(3).value.strip()
                                      ))
                        long.append(dms2dd (coordinate.get(4).value.strip(),
                                       coordinate.get(5).value.strip(),
                                       0,
                                       direction=coordinate.get(6).value.strip()
                                      ))
                    elif len(coordinate.params) == 3:
                        lat.append(float(coordinate.get(1).value.strip()))
                        long.append(float(coordinate.get(2).value.strip()))
                df_row['latitude'] = sum(lat)/len(lat)
                df_row['longitude'] = sum(long)/len(long)
                        
            if template.get("nombre_enlace").value and len(template.get("nombre_enlace").value) > 2:
                df_row["name"] = template.get("nombre_enlace").value.strip()

            if template.get("wikidata").value and len(template.get("wikidata").value) > 2:
                #print template.get("wikidata").value.strip()
                item_page = pb.Page(wikidata_site, title=template.get("wikidata").value.strip())
                try:
                    df_row["wikidata_id"] = template.get("wikidata").value.strip()
                    creation = item_page.oldest_revision
                    df_row["wikidata_timestamp"] = str(epoch_time(creation.timestamp))
                except:
                    pass

            if template.get("imagen").value and len(template.get("imagen").value) > 2:
                df_row["image"] = template.get("imagen").value.strip()

            if template.get("categoría-Commons").value and len(
                    template.get("categoría-Commons").value.strip()) > 3:
                cat_page = pb.Page(commons_site, title=template.get("categoría-Commons").value.strip(), ns=14)
                try:
                    creation = cat_page.oldest_revision
                    df_row["category"] = template.get("categoría-Commons").value.strip()
                    df_row["cat_timestamp"] = str(epoch_time(creation.timestamp))
                except:
                    pass
            festivals_df = festivals_df.append(df_row, ignore_index=True)

In [ ]:
buf = StringIO()
festivals_df.to_csv(buf, index=None, sep=';', encoding='utf-8', header=False)

db_page = pb.Page(commons_site, FESTIVAL_DB_PAGE)
db_page.text = u'<pre>\n' + buf.getvalue() + u'</pre>'
db_page.save("WLF Spain: festival database update")